# Gradient Checking

Welcome to the final assignment for this week! In this assignment you will learn to implement and use gradient checking. 

You are part of a team working to make mobile payments available globally, and are asked to build a deep learning model to detect fraud--whenever someone makes a payment, you want to see if the payment might be fraudulent, such as if the user's account has been taken over by a hacker. 

But backpropagation is quite challenging to implement, and sometimes has bugs. Because this is a mission-critical application, your company's CEO wants to be really certain that your implementation of backpropagation is correct. Your CEO says, "Give me a proof that your backpropagation is actually working!" To give this reassurance, you are going to use "gradient checking".

Let's do it!

In [ ]:
# Packages
import tensorflow as tf
import numpy as np
from testCases import *

# TensorFlow activation functions
def sigmoid(z):
    """Sigmoid activation function using TensorFlow"""
    return tf.nn.sigmoid(z)

def relu(z):
    """ReLU activation function using TensorFlow"""
    return tf.nn.relu(z)

# Utility functions for gradient checking
def dictionary_to_vector(parameters):
    """
    Roll all our parameters dictionary into a single vector satisfying our specific required shape.
    """
    keys = []
    count = 0
    for key in parameters.keys():
        
        # flatten parameter
        new_vector = tf.reshape(parameters[key], (-1,1))
        keys.append(key)
        
        if count == 0:
            theta = new_vector
        else:
            theta = tf.concat([theta, new_vector], axis=0)
        count = count + 1

    return theta, keys

def vector_to_dictionary(theta, keys):
    """
    Unroll all our parameters dictionary from a single vector satisfying our specific required shape.
    """
    parameters = {}
    # Define the shapes for each layer
    shapes = {
        "W1": (5, 4),
        "b1": (5, 1),
        "W2": (3, 5),
        "b2": (3, 1),
        "W3": (1, 3),
        "b3": (1, 1)
    }
    
    start = 0
    for key in keys:
        shape = shapes[key]
        size = shape[0] * shape[1]
        parameters[key] = tf.reshape(theta[start:start+size], shape)
        start += size
    
    return parameters

def gradients_to_vector(gradients):
    """
    Roll all our gradients dictionary into a single vector satisfying our specific required shape.
    """
    count = 0
    # Only include the parameter gradients (W and b), not intermediate activations
    param_keys = ["dW1", "db1", "dW2", "db2", "dW3", "db3"]
    
    for key in param_keys:
        # flatten parameter
        new_vector = tf.reshape(gradients[key], (-1,1))
        
        if count == 0:
            theta = new_vector
        else:
            theta = tf.concat([theta, new_vector], axis=0)
        count = count + 1

    return theta

## TensorFlow Implementation

This notebook has been converted to use **TensorFlow** instead of pure NumPy operations. Key changes include:

- **TensorFlow tensors** instead of NumPy arrays for all computations
- **TensorFlow operations** (`tf.matmul`, `tf.nn.relu`, `tf.nn.sigmoid`, etc.) instead of NumPy equivalents
- **Automatic differentiation** capabilities (though we still implement manual gradient checking for educational purposes)
- **GPU acceleration** support (if available)
- **More numerically stable** operations (e.g., `tf.nn.sigmoid_cross_entropy_with_logits`)

The core concepts of gradient checking remain the same, but now we're using TensorFlow's computational graph and tensor operations throughout.

## 1) How does gradient checking work?

Backpropagation computes the gradients $\frac{\partial J}{\partial \theta}$, where $\theta$ denotes the parameters of the model. $J$ is computed using forward propagation and your loss function.

Because forward propagation is relatively easy to implement, you're confident you got that right, and so you're almost  100% sure that you're computing the cost $J$ correctly. Thus, you can use your code for computing $J$ to verify the code for computing $\frac{\partial J}{\partial \theta}$. 

Let's look back at the definition of a derivative (or gradient):
$$ \frac{\partial J}{\partial \theta} = \lim_{\varepsilon \to 0} \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon} \tag{1}$$

If you're not familiar with the "$\displaystyle \lim_{\varepsilon \to 0}$" notation, it's just a way of saying "when $\varepsilon$ is really really small."

We know the following:

- $\frac{\partial J}{\partial \theta}$ is what you want to make sure you're computing correctly. 
- You can compute $J(\theta + \varepsilon)$ and $J(\theta - \varepsilon)$ (in the case that $\theta$ is a real number), since you're confident your implementation for $J$ is correct. 

Lets use equation (1) and a small value for $\varepsilon$ to convince your CEO that your code for computing  $\frac{\partial J}{\partial \theta}$ is correct!

## 2) 1-dimensional gradient checking

Consider a 1D linear function $J(\theta) = \theta x$. The model contains only a single real-valued parameter $\theta$, and takes $x$ as input.

You will implement code to compute $J(.)$ and its derivative $\frac{\partial J}{\partial \theta}$. You will then use gradient checking to make sure your derivative computation for $J$ is correct. 

<img src="images/1Dgrad_kiank.png" style="width:600px;height:250px;">
<caption><center> <u> **Figure 1** </u>: **1D linear model**<br> </center></caption>

The diagram above shows the key computation steps: First start with $x$, then evaluate the function $J(x)$ ("forward propagation"). Then compute the derivative $\frac{\partial J}{\partial \theta}$ ("backward propagation"). 

**Exercise**: implement "forward propagation" and "backward propagation" for this simple function. I.e., compute both $J(.)$ ("forward propagation") and its derivative with respect to $\theta$ ("backward propagation"), in two separate functions. 

In [ ]:
import tensorflow as tf

# GRADED FUNCTION: forward_propagation

def forward_propagation(x, theta):
    """
    Implement the linear forward propagation (compute J) presented in Figure 1 (J(theta) = theta * x)
    Using TensorFlow operations
    
    Arguments:
    x -- a real-valued input (can be a TensorFlow tensor or Python number)
    theta -- our parameter, a real number as well (can be a TensorFlow tensor or Python number)
    
    Returns:
    J -- the value of function J, computed using the formula J(theta) = theta * x
    """
    
    ### START CODE HERE ### (approx. 1 line)
    # Convert inputs to TensorFlow tensors if they aren't already
    x_tensor = tf.constant(x, dtype=tf.float32)
    theta_tensor = tf.constant(theta, dtype=tf.float32)
    J = tf.multiply(theta_tensor, x_tensor)
    ### END CODE HERE ###
    
    return J

In [5]:
x, theta = 2, 4
J = forward_propagation(x, theta)
print ("J = " + str(J))

J = 8


**Expected Output**:

<table style=>
    <tr>
        <td>  ** J **  </td>
        <td> 8</td>
    </tr>
</table>

**Exercise**: Now, implement the backward propagation step (derivative computation) of Figure 1. That is, compute the derivative of $J(\theta) = \theta x$ with respect to $\theta$. To save you from doing the calculus, you should get $dtheta = \frac { \partial J }{ \partial \theta} = x$.

In [ ]:
import tensorflow as tf

# GRADED FUNCTION: backward_propagation

def backward_propagation(x, theta):
    """
    Computes the derivative of J with respect to theta (see Figure 1).
    Using TensorFlow operations
    
    Arguments:
    x -- a real-valued input (can be a TensorFlow tensor or Python number)
    theta -- our parameter, a real number as well (can be a TensorFlow tensor or Python number)
    
    Returns:
    dtheta -- the gradient of the cost with respect to theta
    """
    
    ### START CODE HERE ### (approx. 1 line)
    # For J(theta) = theta * x, the derivative is simply x
    dtheta = tf.constant(x, dtype=tf.float32)
    ### END CODE HERE ###
    
    return dtheta

In [7]:
x, theta = 2, 4
dtheta = backward_propagation(x, theta)
print ("dtheta = " + str(dtheta))

dtheta = 2


**Expected Output**:

<table>
    <tr>
        <td>  ** dtheta **  </td>
        <td> 2 </td>
    </tr>
</table>

**Exercise**: To show that the `backward_propagation()` function is correctly computing the gradient $\frac{\partial J}{\partial \theta}$, let's implement gradient checking.

**Instructions**:
- First compute "gradapprox" using the formula above (1) and a small value of $\varepsilon$. Here are the Steps to follow:
    1. $\theta^{+} = \theta + \varepsilon$
    2. $\theta^{-} = \theta - \varepsilon$
    3. $J^{+} = J(\theta^{+})$
    4. $J^{-} = J(\theta^{-})$
    5. $gradapprox = \frac{J^{+} - J^{-}}{2  \varepsilon}$
- Then compute the gradient using backward propagation, and store the result in a variable "grad"
- Finally, compute the relative difference between "gradapprox" and the "grad" using the following formula:
$$ difference = \frac {\mid\mid grad - gradapprox \mid\mid_2}{\mid\mid grad \mid\mid_2 + \mid\mid gradapprox \mid\mid_2} \tag{2}$$
You will need 3 Steps to compute this formula:
   - 1'. compute the numerator using np.linalg.norm(...)
   - 2'. compute the denominator. You will need to call np.linalg.norm(...) twice.
   - 3'. divide them.
- If this difference is small (say less than $10^{-7}$), you can be quite confident that you have computed your gradient correctly. Otherwise, there may be a mistake in the gradient computation. 


In [ ]:
import numpy as np
import tensorflow as tf

# GRADED FUNCTION: gradient_check

def gradient_check(x, theta, epsilon = 1e-7):
    """
    Implement the backward propagation presented in Figure 1.
    Using TensorFlow operations for gradient checking
    
    Arguments:
    x -- a real-valued input
    theta -- our parameter, a real number as well
    epsilon -- tiny shift to the input to compute approximated gradient with formula(1)
    
    Returns:
    difference -- difference (2) between the approximated gradient and the backward propagation gradient
    """
    
    # Compute gradapprox using left side of formula (1). epsilon is small enough, you don't need to worry about the limit.
    ### START CODE HERE ### (approx. 5 lines)
    # Convert to TensorFlow tensors
    theta_tensor = tf.constant(theta, dtype=tf.float32)
    x_tensor = tf.constant(x, dtype=tf.float32)
    epsilon_tensor = tf.constant(epsilon, dtype=tf.float32)
    
    thetaplus = theta_tensor + epsilon_tensor                        # Step 1
    thetaminus = theta_tensor - epsilon_tensor                       # Step 2
    J_plus = tf.multiply(thetaplus, x_tensor)                        # Step 3
    J_minus = tf.multiply(thetaminus, x_tensor)                      # Step 4
    gradapprox = (J_plus - J_minus) / (2 * epsilon_tensor)          # Step 5
    ### END CODE HERE ###
    
    # Check if gradapprox is close enough to the output of backward_propagation()
    ### START CODE HERE ### (approx. 1 line)
    grad = tf.constant(x, dtype=tf.float32)
    ### END CODE HERE ###
    
    ### START CODE HERE ### (approx. 1 line)
    numerator = tf.norm(gradapprox - grad)                           # Step 1'
    denominator = tf.norm(gradapprox) + tf.norm(grad)                # Step 2'
    difference = numerator / denominator                             # Step 3'
    ### END CODE HERE ###
    
    # Convert to numpy for comparison
    difference_value = difference.numpy()
    
    if difference_value < 1e-7:
        print ("The gradient is correct!")
    else:
        print ("The gradient is wrong!")
    
    return difference_value

In [9]:
x, theta = 2, 4
difference = gradient_check(x, theta)
print("difference = " + str(difference))

The gradient is correct!
difference = 2.919335883291695e-10


**Expected Output**:
The gradient is correct!
<table>
    <tr>
        <td>  ** difference **  </td>
        <td> 2.9193358103083e-10 </td>
    </tr>
</table>

Congrats, the difference is smaller than the $10^{-7}$ threshold. So you can have high confidence that you've correctly computed the gradient in `backward_propagation()`. 

Now, in the more general case, your cost function $J$ has more than a single 1D input. When you are training a neural network, $\theta$ actually consists of multiple matrices $W^{[l]}$ and biases $b^{[l]}$! It is important to know how to do a gradient check with higher-dimensional inputs. Let's do it!

## 3) N-dimensional gradient checking

The following figure describes the forward and backward propagation of your fraud detection model.

<img src="images/NDgrad_kiank.png" style="width:600px;height:400px;">
<caption><center> <u> **Figure 2** </u>: **deep neural network**<br>*LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID*</center></caption>

Let's look at your implementations for forward propagation and backward propagation. 

In [ ]:
import tensorflow as tf

def forward_propagation_n(X, Y, parameters):
    """
    Implements the forward propagation (and computes the cost) presented in Figure 3.
    Using TensorFlow operations
    
    Arguments:
    X -- training set for m examples
    Y -- labels for m examples 
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3":
                    W1 -- weight matrix of shape (5, 4)
                    b1 -- bias vector of shape (5, 1)
                    W2 -- weight matrix of shape (3, 5)
                    b2 -- bias vector of shape (3, 1)
                    W3 -- weight matrix of shape (1, 3)
                    b3 -- bias vector of shape (1, 1)
    
    Returns:
    cost -- the cost function (logistic cost for one example)
    """
    
    # Convert inputs to TensorFlow tensors
    X = tf.constant(X, dtype=tf.float32)
    Y = tf.constant(Y, dtype=tf.float32)
    
    # retrieve parameters and convert to TensorFlow tensors
    m = tf.cast(tf.shape(X)[1], tf.float32)
    W1 = tf.constant(parameters["W1"], dtype=tf.float32)
    b1 = tf.constant(parameters["b1"], dtype=tf.float32)
    W2 = tf.constant(parameters["W2"], dtype=tf.float32)
    b2 = tf.constant(parameters["b2"], dtype=tf.float32)
    W3 = tf.constant(parameters["W3"], dtype=tf.float32)
    b3 = tf.constant(parameters["b3"], dtype=tf.float32)

    # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
    Z1 = tf.matmul(W1, X) + b1
    A1 = tf.nn.relu(Z1)
    Z2 = tf.matmul(W2, A1) + b2
    A2 = tf.nn.relu(Z2)
    Z3 = tf.matmul(W3, A2) + b3
    A3 = tf.nn.sigmoid(Z3)

    # Cost - using TensorFlow's built-in functions for numerical stability
    # Equivalent to: logprobs = -Y * log(A3) - (1-Y) * log(1-A3)
    logprobs = tf.nn.sigmoid_cross_entropy_with_logits(logits=Z3, labels=Y)
    cost = tf.reduce_mean(logprobs)
    
    # Convert tensors back to numpy for cache (to maintain compatibility)
    cache = (Z1.numpy(), A1.numpy(), W1.numpy(), b1.numpy(), 
             Z2.numpy(), A2.numpy(), W2.numpy(), b2.numpy(), 
             Z3.numpy(), A3.numpy(), W3.numpy(), b3.numpy())
    
    return cost.numpy(), cache

Now, run backward propagation.

In [ ]:
import tensorflow as tf

def backward_propagation_n(X, Y, cache):
    """
    Implement the backward propagation presented in figure 2.
    Using TensorFlow operations for automatic differentiation
    
    Arguments:
    X -- input datapoint, of shape (input size, 1)
    Y -- true "label"
    cache -- cache output from forward_propagation_n()
    
    Returns:
    gradients -- A dictionary with the gradients of the cost with respect to each parameter, activation and pre-activation variables.
    """
    
    # Convert inputs to TensorFlow tensors
    X = tf.constant(X, dtype=tf.float32)
    Y = tf.constant(Y, dtype=tf.float32)
    
    m = tf.cast(tf.shape(X)[1], tf.float32)
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    # Convert cache to TensorFlow tensors
    Z1 = tf.constant(Z1, dtype=tf.float32)
    A1 = tf.constant(A1, dtype=tf.float32)
    W1 = tf.constant(W1, dtype=tf.float32)
    b1 = tf.constant(b1, dtype=tf.float32)
    Z2 = tf.constant(Z2, dtype=tf.float32)
    A2 = tf.constant(A2, dtype=tf.float32)
    W2 = tf.constant(W2, dtype=tf.float32)
    b2 = tf.constant(b2, dtype=tf.float32)
    Z3 = tf.constant(Z3, dtype=tf.float32)
    A3 = tf.constant(A3, dtype=tf.float32)
    W3 = tf.constant(W3, dtype=tf.float32)
    b3 = tf.constant(b3, dtype=tf.float32)
    
    # Backward propagation using TensorFlow operations
    dZ3 = A3 - Y
    dW3 = (1.0/m) * tf.matmul(dZ3, tf.transpose(A2))
    db3 = (1.0/m) * tf.reduce_sum(dZ3, axis=1, keepdims=True)
    
    dA2 = tf.matmul(tf.transpose(W3), dZ3)
    dZ2 = tf.multiply(dA2, tf.cast(A2 > 0, tf.float32))  # ReLU derivative
    dW2 = (1.0/m) * tf.matmul(dZ2, tf.transpose(A1))
    db2 = (1.0/m) * tf.reduce_sum(dZ2, axis=1, keepdims=True)
    
    dA1 = tf.matmul(tf.transpose(W2), dZ2)
    dZ1 = tf.multiply(dA1, tf.cast(A1 > 0, tf.float32))  # ReLU derivative
    dW1 = (1.0/m) * tf.matmul(dZ1, tf.transpose(X))
    db1 = (1.0/m) * tf.reduce_sum(dZ1, axis=1, keepdims=True)
    
    # Convert back to numpy for compatibility
    gradients = {"dZ3": dZ3.numpy(), "dW3": dW3.numpy(), "db3": db3.numpy(),
                 "dA2": dA2.numpy(), "dZ2": dZ2.numpy(), "dW2": dW2.numpy(), "db2": db2.numpy(),
                 "dA1": dA1.numpy(), "dZ1": dZ1.numpy(), "dW1": dW1.numpy(), "db1": db1.numpy()}
    
    return gradients

You obtained some results on the fraud detection test set but you are not 100% sure of your model. Nobody's perfect! Let's implement gradient checking to verify if your gradients are correct.

**How does gradient checking work?**.

As in 1) and 2), you want to compare "gradapprox" to the gradient computed by backpropagation. The formula is still:

$$ \frac{\partial J}{\partial \theta} = \lim_{\varepsilon \to 0} \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon} \tag{1}$$

However, $\theta$ is not a scalar anymore. It is a dictionary called "parameters". We implemented a function "`dictionary_to_vector()`" for you. It converts the "parameters" dictionary into a vector called "values", obtained by reshaping all parameters (W1, b1, W2, b2, W3, b3) into vectors and concatenating them.

The inverse function is "`vector_to_dictionary`" which outputs back the "parameters" dictionary.

<img src="images/dictionary_to_vector.png" style="width:600px;height:400px;">
<caption><center> <u> **Figure 2** </u>: **dictionary_to_vector() and vector_to_dictionary()**<br> You will need these functions in gradient_check_n()</center></caption>

We have also converted the "gradients" dictionary into a vector "grad" using gradients_to_vector(). You don't need to worry about that.

**Exercise**: Implement gradient_check_n().

**Instructions**: Here is pseudo-code that will help you implement the gradient check.

For each i in num_parameters:
- To compute `J_plus[i]`:
    1. Set $\theta^{+}$ to `np.copy(parameters_values)`
    2. Set $\theta^{+}_i$ to $\theta^{+}_i + \varepsilon$
    3. Calculate $J^{+}_i$ using to `forward_propagation_n(x, y, vector_to_dictionary(`$\theta^{+}$ `))`.     
- To compute `J_minus[i]`: do the same thing with $\theta^{-}$
- Compute $gradapprox[i] = \frac{J^{+}_i - J^{-}_i}{2 \varepsilon}$

Thus, you get a vector gradapprox, where gradapprox[i] is an approximation of the gradient with respect to `parameter_values[i]`. You can now compare this gradapprox vector to the gradients vector from backpropagation. Just like for the 1D case (Steps 1', 2', 3'), compute: 
$$ difference = \frac {\| grad - gradapprox \|_2}{\| grad \|_2 + \| gradapprox \|_2 } \tag{3}$$

In [ ]:
# GRADED FUNCTION: gradient_check_n

def gradient_check_n(parameters, gradients, X, Y, epsilon = 1e-7):
    """
    Checks if backward_propagation_n computes correctly the gradient of the cost output by forward_propagation_n
    Using TensorFlow operations
    
    Arguments:
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3":
    grad -- output of backward_propagation_n, contains gradients of the cost with respect to the parameters. 
    x -- input datapoint, of shape (input size, 1)
    y -- true "label"
    epsilon -- tiny shift to the input to compute approximated gradient with formula(1)
    
    Returns:
    difference -- difference (2) between the approximated gradient and the backward propagation gradient
    """
    
    # Set-up variables
    parameters_values, keys = dictionary_to_vector(parameters)
    grad = gradients_to_vector(gradients)
    num_parameters = tf.shape(parameters_values)[0]
    J_plus = tf.zeros((num_parameters, 1), dtype=tf.float32)
    J_minus = tf.zeros((num_parameters, 1), dtype=tf.float32)
    gradapprox = tf.zeros((num_parameters, 1), dtype=tf.float32)
    
    # Convert to TensorFlow tensors
    epsilon_tensor = tf.constant(epsilon, dtype=tf.float32)
    
    # Convert parameters_values to numpy for easier manipulation
    parameters_values_np = parameters_values.numpy()
    J_plus_list = []
    J_minus_list = []
    
    # Compute gradapprox
    for i in range(num_parameters.numpy()):
        
        # Compute J_plus[i]. Inputs: "parameters_values, epsilon". Output = "J_plus[i]".
        # "_" is used because the function you have to outputs two parameters but we only care about the first one
        ### START CODE HERE ### (approx. 3 lines)
        thetaplus = np.copy(parameters_values_np)                                        # Step 1
        thetaplus[i][0] = thetaplus[i][0] + epsilon                                     # Step 2
        thetaplus_tensor = tf.constant(thetaplus, dtype=tf.float32)
        J_plus_i, _ = forward_propagation_n(X, Y, vector_to_dictionary(thetaplus_tensor, keys))  # Step 3
        J_plus_list.append(J_plus_i)
        ### END CODE HERE ###
        
        # Compute J_minus[i]. Inputs: "parameters_values, epsilon". Output = "J_minus[i]".
        ### START CODE HERE ### (approx. 3 lines)
        thetaminus = np.copy(parameters_values_np)                                       # Step 1
        thetaminus[i][0] = thetaminus[i][0] - epsilon                                   # Step 2
        thetaminus_tensor = tf.constant(thetaminus, dtype=tf.float32)
        J_minus_i, _ = forward_propagation_n(X, Y, vector_to_dictionary(thetaminus_tensor, keys)) # Step 3
        J_minus_list.append(J_minus_i)
        ### END CODE HERE ###
    
    # Convert lists to TensorFlow tensors
    J_plus = tf.constant([[val] for val in J_plus_list], dtype=tf.float32)
    J_minus = tf.constant([[val] for val in J_minus_list], dtype=tf.float32)
    
    # Compute gradapprox
    ### START CODE HERE ### (approx. 1 line)
    gradapprox = (J_plus - J_minus) / (2 * epsilon_tensor)
    ### END CODE HERE ###
    
    # Compare gradapprox to backward propagation gradients by computing difference.
    ### START CODE HERE ### (approx. 1 line)
    numerator = tf.norm(grad - gradapprox)                          # Step 1'
    denominator = tf.norm(grad) + tf.norm(gradapprox)               # Step 2'
    difference = numerator / denominator                            # Step 3'
    ### END CODE HERE ###

    # Convert to numpy for printing
    difference_value = difference.numpy()
    
    if difference_value > 2e-7:
        print ("\033[93m" + "There is a mistake in the backward propagation! difference = " + str(difference_value) + "\033[0m")
    else:
        print ("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference_value) + "\033[0m")
    
    return difference_value

In [21]:
X, Y, parameters = gradient_check_n_test_case()

cost, cache = forward_propagation_n(X, Y, parameters)
gradients = backward_propagation_n(X, Y, cache)
difference = gradient_check_n(parameters, gradients, X, Y)

Your backward propagation works perfectly fine! difference = 1.1890913023330276e-07


**Expected output**:

<table>
    <tr>
        <td>  ** There is a mistake in the backward propagation!**  </td>
        <td> difference = 0.285093156781 </td>
    </tr>
</table>

It seems that there were errors in the `backward_propagation_n` code we gave you! Good that you've implemented the gradient check. Go back to `backward_propagation` and try to find/correct the errors *(Hint: check dW2 and db1)*. Rerun the gradient check when you think you've fixed it. Remember you'll need to re-execute the cell defining `backward_propagation_n()` if you modify the code. 

Can you get gradient check to declare your derivative computation correct? Even though this part of the assignment isn't graded, we strongly urge you to try to find the bug and re-run gradient check until you're convinced backprop is now correctly implemented. 

**Note** 
- Gradient Checking is slow! Approximating the gradient with $\frac{\partial J}{\partial \theta} \approx  \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon}$ is computationally costly. For this reason, we don't run gradient checking at every iteration during training. Just a few times to check if the gradient is correct. 
- Gradient Checking, at least as we've presented it, doesn't work with dropout. You would usually run the gradient check algorithm without dropout to make sure your backprop is correct, then add dropout. 

Congrats, you can be confident that your deep learning model for fraud detection is working correctly! You can even use this to convince your CEO. :) 

<font color='blue'>
**What you should remember from this notebook**:
- Gradient checking verifies closeness between the gradients from backpropagation and the numerical approximation of the gradient (computed using forward propagation).
- Gradient checking is slow, so we don't run it in every iteration of training. You would usually run it only to make sure your code is correct, then turn it off and use backprop for the actual learning process. 

## Bonus: TensorFlow's Automatic Differentiation

While gradient checking is essential for verifying manual backpropagation implementations, TensorFlow provides automatic differentiation through `tf.GradientTape`. Let's see how this compares to our manual implementation:

In [ ]:
def forward_propagation_with_autodiff(X, Y, parameters):
    """
    Demonstrates TensorFlow's automatic differentiation for the same network
    """
    # Convert parameters to TensorFlow Variables (required for gradient computation)
    W1 = tf.Variable(parameters["W1"], dtype=tf.float32)
    b1 = tf.Variable(parameters["b1"], dtype=tf.float32)
    W2 = tf.Variable(parameters["W2"], dtype=tf.float32)
    b2 = tf.Variable(parameters["b2"], dtype=tf.float32)
    W3 = tf.Variable(parameters["W3"], dtype=tf.float32)
    b3 = tf.Variable(parameters["b3"], dtype=tf.float32)
    
    X = tf.constant(X, dtype=tf.float32)
    Y = tf.constant(Y, dtype=tf.float32)
    
    # Use GradientTape to record operations for automatic differentiation
    with tf.GradientTape() as tape:
        # Forward propagation
        Z1 = tf.matmul(W1, X) + b1
        A1 = tf.nn.relu(Z1)
        Z2 = tf.matmul(W2, A1) + b2
        A2 = tf.nn.relu(Z2)
        Z3 = tf.matmul(W3, A2) + b3
        A3 = tf.nn.sigmoid(Z3)
        
        # Cost computation
        cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Z3, labels=Y))
    
    # Compute gradients automatically
    variables = [W1, b1, W2, b2, W3, b3]
    gradients = tape.gradient(cost, variables)
    
    # Convert gradients to dictionary format
    grad_dict = {
        "dW1": gradients[0].numpy(),
        "db1": gradients[1].numpy(),
        "dW2": gradients[2].numpy(),
        "db2": gradients[3].numpy(),
        "dW3": gradients[4].numpy(),
        "db3": gradients[5].numpy()
    }
    
    return cost.numpy(), grad_dict

# Example usage (uncomment to run after loading test data):
# X, Y, parameters = gradient_check_n_test_case()
# cost_auto, gradients_auto = forward_propagation_with_autodiff(X, Y, parameters)
# cost_manual, cache = forward_propagation_n(X, Y, parameters)
# gradients_manual = backward_propagation_n(X, Y, cache)
# 
# print("Cost (auto-diff):", cost_auto)
# print("Cost (manual):", cost_manual)
# print("Gradient difference (dW1):", np.max(np.abs(gradients_auto["dW1"] - gradients_manual["dW1"])))

## Benefits of Using TensorFlow

**Key advantages of this TensorFlow implementation:**

1. **Automatic Differentiation**: TensorFlow can compute gradients automatically using `tf.GradientTape`, eliminating the need for manual backpropagation (though gradient checking is still valuable for verification).

2. **GPU Acceleration**: All operations can run on GPU with minimal code changes, providing significant speedup for large networks.

3. **Numerical Stability**: TensorFlow provides numerically stable implementations of common operations (e.g., `tf.nn.sigmoid_cross_entropy_with_logits`).

4. **Production Ready**: TensorFlow is designed for production deployment with features like model serving, optimization, and quantization.

5. **Ecosystem**: Rich ecosystem with Keras for high-level model building, TensorBoard for visualization, and TensorFlow Lite for mobile deployment.

**When to use gradient checking:**
- When implementing custom layers or loss functions
- When debugging training issues
- When developing new optimization algorithms
- For educational purposes to understand backpropagation

Remember: Gradient checking is computationally expensive and should only be used during development, not during actual training!